In [20]:
from sklearn.metrics import mean_squared_error
from sklearn.linear_model import LinearRegression
import pandas as pd 
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import os 

In [21]:
train_df = pd.read_csv('train.csv')

In [22]:
train_df

,id,date,year_week,product_number,reporterhq_id,prod_category,specs,display_size,segment,sales_units,inventory_units
0,202119-6909,2021-05-15,202119,6909,93,Arale,21274,13.3,Premium,2.0,35.0
1,202120-6909,2021-05-22,202120,6909,93,Arale,21274,13.3,Premium,0.0,70.0
2,202121-6909,2021-05-29,202121,6909,93,Arale,21274,13.3,Premium,3.0,137.0
3,202122-6909,2021-06-05,202122,6909,93,Arale,21274,13.3,Premium,0.0,274.0
4,202123-6909,2021-06-12,202123,6909,93,Arale,21274,13.3,Premium,0.0,333.0
...,...,...,...,...,...,...,...,...,...,...,...
15449,202317-247737,2023-04-29,202317,247737,15,Arale,10637,13.3,Premium,0.0,62.0
15450,202317-247737,2023-04-29,202317,247737,24,Arale,10637,13.3,Premium,0.0,3.0
15451,202318-247737,2023-05-06,202318,247737,3,Arale,10637,13.3,Premium,22.0,0.0
15452,202318-247737,2023-05-06,202318,247737,15,Arale,10637,13.3,Premium,68.0,78.0


In [23]:
train_df = train_df.drop(columns='sales_units')

In [24]:
train_df = train_df.dropna()

In [25]:
train_df['date'] = pd.to_datetime(train_df['date'])

In [26]:
train_df = train_df.groupby(['date', 'product_number']).agg({'inventory_units': 'sum', 'reporterhq_id': 'first', 'prod_category': 'first', 'specs': 'first', 'display_size': 'first', 'segment': 'first'}).reset_index()
train_df.drop(columns='reporterhq_id', inplace=True)

In [27]:
train_df

,date,product_number,inventory_units,prod_category,specs,display_size,segment
0,2019-04-13,233919,20.0,Clover,106370,15.6,Core
1,2019-04-20,233919,43.0,Clover,106370,15.6,Core
2,2019-04-27,233919,184.0,Clover,106370,15.6,Core
3,2019-05-04,233919,219.0,Clover,106370,15.6,Core
4,2019-05-11,233919,133.0,Clover,106370,15.6,Core
...,...,...,...,...,...,...,...
7620,2023-05-06,238854,122.0,Goku,117007,15.6,Core
7621,2023-05-06,239841,36.0,Goku,95733,15.6,Core
7622,2023-05-06,240828,79.0,Conan,2901,13.3,Premium
7623,2023-05-06,242802,57.0,Conan,3868,13.3,Premium


## Modeling

In [28]:
def prepare_df(df):
    df = df[['date', 'product_number', 'prod_category', 'segment', 'specs', 'display_size', 'inventory_units']]

    df['year'] = df['date'].dt.year
    df['month'] = df['date'].dt.month
    df['week'] = df['date'].dt.week
    df['quarter'] = df['date'].dt.quarter

    df = pd.get_dummies(df, columns=['segment', 'prod_category', 'year', 'month', 'week', 'quarter'])

    return df

In [29]:
final_df = prepare_df(train_df.copy()).sort_values(by='date').reset_index(drop=True)

/var/folders/c6/kf2fcpcj6pq_gwpfbfzzy81m0000gn/T/ipykernel_69487/567409335.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['year'] = df['date'].dt.year
/var/folders/c6/kf2fcpcj6pq_gwpfbfzzy81m0000gn/T/ipykernel_69487/567409335.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['month'] = df['date'].dt.month
/var/folders/c6/kf2fcpcj6pq_gwpfbfzzy81m0000gn/T/ipykernel_69487/567409335.py:6: FutureWarning: Series.dt.weekofyear and Series.dt.week have been deprecated. Please use Series.dt.isocalendar()

In [30]:
from xgboost import XGBRegressor
from catboost import CatBoostRegressor
from sklearn.linear_model import Ridge, ElasticNet
from window_ops.rolling import rolling_mean
from window_ops.expanding import expanding_mean
from window_ops.ewm import ewm_mean
from mlforecast import MLForecast
from mlforecast.target_transforms import Differences

models = [
    XGBRegressor(),
    CatBoostRegressor(verbose=0),
    Ridge(),
    ElasticNet(),
]

model = MLForecast(models=models,
                   freq='W',
                   lags=[1,2,3,4,5,8,12],
                   lag_transforms={
                        1: [(ewm_mean, 0.9), (rolling_mean, 2), (rolling_mean, 4)],
                        4: [expanding_mean, (rolling_mean, 4)],
                        12: [(rolling_mean, 2)],
                   },
                   num_threads=8,
                   target_transforms=[Differences([1])]
)

In [31]:
model.fit(
    final_df,
    id_col='product_number', 
    time_col='date', 
    target_col='inventory_units', 
)

/Users/alexserrano/miniforge3/lib/python3.10/site-packages/sklearn/linear_model/_coordinate_descent.py:631: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 5.833e+03, tolerance: 2.643e+03
  model = cd_fast.enet_coordinate_descent(


MLForecast(models=[XGBRegressor, CatBoostRegressor, Ridge, ElasticNet], freq=<Week: weekday=6>, lag_features=['lag1', 'lag2', 'lag3', 'lag4', 'lag5', 'lag8', 'lag12', 'ewm_mean_lag1_alpha0.9', 'rolling_mean_lag1_window_size2', 'rolling_mean_lag1_window_size4', 'expanding_mean_lag4', 'rolling_mean_lag4_window_size4', 'rolling_mean_lag12_window_size2'], date_features=[], num_threads=8)

In [32]:
final_preds = model.predict(13)
final_preds

,product_number,date,XGBRegressor,CatBoostRegressor,Ridge,ElasticNet
0,6909,2023-05-07,82.464882,89.792903,78.321079,91.986061
1,6909,2023-05-14,77.720810,91.647629,68.901180,95.274332
2,6909,2023-05-21,79.984436,99.032718,62.417039,99.193814
3,6909,2023-05-28,84.194740,106.414538,58.965996,103.679360
4,6909,2023-06-04,89.315598,109.209959,59.031581,108.874308
...,...,...,...,...,...,...
1295,247737,2023-07-02,8.089865,19.140891,31.047224,162.391075
1296,247737,2023-07-09,4.432044,7.713132,3.075919,161.348342
1297,247737,2023-07-16,26.074039,-2.384170,-29.074183,155.219703
1298,247737,2023-07-23,34.458633,-14.003285,-62.826311,150.989306


In [33]:
# ID => YYYYWW-PRODUCT_NUMBER
final_preds['date'] = pd.to_datetime(final_preds['date'])
final_preds['week'] = final_preds['date'].dt.week
final_preds['year'] = final_preds['date'].dt.year
final_preds['id'] = final_preds['year'].astype(str) + final_preds['week'].astype(str) + '-' + final_preds['product_number'].astype(str)

final_preds['mean_ridge_catboost'] = (final_preds['Ridge'] + final_preds['CatBoostRegressor']) / 2

/var/folders/c6/kf2fcpcj6pq_gwpfbfzzy81m0000gn/T/ipykernel_69487/2531016594.py:3: FutureWarning: Series.dt.weekofyear and Series.dt.week have been deprecated. Please use Series.dt.isocalendar().week instead.
  final_preds['week'] = final_preds['date'].dt.week


In [34]:
final_preds = final_preds.sort_values(by=['id'])
final_preds

,product_number,date,XGBRegressor,CatBoostRegressor,Ridge,ElasticNet,week,year,id,mean_ridge_catboost
546,105609,2023-05-07,86.181763,91.948190,94.817334,87.828603,18,2023,202318-105609,93.382762
559,107583,2023-05-07,236.462753,247.605024,241.041646,228.298200,18,2023,202318-107583,244.323335
52,10857,2023-05-07,228.298218,226.137468,207.297196,212.491261,18,2023,202318-10857,216.717332
572,109557,2023-05-07,90.192078,97.238591,98.953461,101.935332,18,2023,202318-109557,98.096026
585,112518,2023-05-07,165.685486,182.830131,195.814579,200.267414,18,2023,202318-112518,189.322355
...,...,...,...,...,...,...,...,...,...,...
506,92778,2023-07-30,411.210083,397.722364,358.644081,34.869309,30,2023,202330-92778,378.183222
519,93765,2023-07-30,314.779877,352.391490,191.490054,107.321078,30,2023,202330-93765,271.940772
532,94752,2023-07-30,654.088257,587.808974,567.949431,360.692775,30,2023,202330-94752,577.879202
51,9870,2023-07-30,418.122070,391.440905,-124.580504,91.356523,30,2023,202330-9870,133.430200


In [35]:
test_df = pd.read_csv('test.csv')
test_df

,id
0,202319-105609
1,202319-107583
2,202319-10857
3,202319-109557
4,202319-112518
...,...
1295,202331-92778
1296,202331-93765
1297,202331-94752
1298,202331-9870


In [36]:
final_preds

,product_number,date,XGBRegressor,CatBoostRegressor,Ridge,ElasticNet,week,year,id,mean_ridge_catboost
546,105609,2023-05-07,86.181763,91.948190,94.817334,87.828603,18,2023,202318-105609,93.382762
559,107583,2023-05-07,236.462753,247.605024,241.041646,228.298200,18,2023,202318-107583,244.323335
52,10857,2023-05-07,228.298218,226.137468,207.297196,212.491261,18,2023,202318-10857,216.717332
572,109557,2023-05-07,90.192078,97.238591,98.953461,101.935332,18,2023,202318-109557,98.096026
585,112518,2023-05-07,165.685486,182.830131,195.814579,200.267414,18,2023,202318-112518,189.322355
...,...,...,...,...,...,...,...,...,...,...
506,92778,2023-07-30,411.210083,397.722364,358.644081,34.869309,30,2023,202330-92778,378.183222
519,93765,2023-07-30,314.779877,352.391490,191.490054,107.321078,30,2023,202330-93765,271.940772
532,94752,2023-07-30,654.088257,587.808974,567.949431,360.692775,30,2023,202330-94752,577.879202
51,9870,2023-07-30,418.122070,391.440905,-124.580504,91.356523,30,2023,202330-9870,133.430200


In [37]:
submission = pd.DataFrame()
submission['id'] = test_df['id']
submission['inventory_units'] = final_preds['mean_ridge_catboost'].values

submission

,id,inventory_units
0,202319-105609,93.382762
1,202319-107583,244.323335
2,202319-10857,216.717332
3,202319-109557,98.096026
4,202319-112518,189.322355
...,...,...
1295,202331-92778,378.183222
1296,202331-93765,271.940772
1297,202331-94752,577.879202
1298,202331-9870,133.430200


In [38]:
submission.to_csv('submission_estim.csv', index = False)